### Model VAERS Dataset

In [1]:
dataset_id = "vaers_modeled"
!bq --location=US mk --dataset {dataset_id}

Dataset 'studied-brand-266702:vaers_modeled' successfully created.


### Split staging Events table into separate entities: Patient table and Adverse_Event table 

#### Create Patient table

In [20]:
%%bigquery
# create table vaers_modeled.Patient as
select DISTINCT ROW_NUMBER() OVER() as PATIENT_ID, VAERS_ID, STATE, AGE_YRS, SEX, ALLERGIES
from vaers_staging.Events

,PATIENT_ID,VAERS_ID,STATE,AGE_YRS,SEX,ALLERGIES
0,14,803845,WV,65.0,F,dexlansoprazole
1,45,820113,DE,67.0,U,Sulphate drugs Penicillin
2,123,794191,MS,NaN,M,None
3,143,794340,AR,NaN,M,None
4,239,794809,WV,NaN,M,None
...,...,...,...,...,...,...
44339,44141,819290,WI,89.0,F,clindamycin cephalexin
44340,44184,843628,WI,89.0,F,none
44341,44209,815089,WI,90.0,F,None
44342,44233,824511,WI,90.0,F,none


#### Create Adverse_Event table

In [13]:
%%bigquery
# create table vaers_modeled.Adverse_Event as
select VAERS_ID, ONSET_DATE, RECOVD, DIED, DATEDIED, L_THREAT, OFC_VISIT, ER_VISIT, ER_ED_VISIT, HOSPITAL, HOSPDAYS, X_STAY, DISABLE, BIRTH_DEFECT, OTHER_MEDS, CUR_ILL, HISTORY, PRIOR_VAX 
from vaers_staging.Events

,VAERS_ID,ONSET_DATE,RECOVD,DIED,DATEDIED,L_THREAT,OFC_VISIT,ER_VISIT,ER_ED_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,BIRTH_DEFECT,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX
0,794224,2018-12-29,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None
1,794226,2018-12-28,None,None,None,None,None,None,None,None,NaN,None,None,None,WELLBUTRIN; omeprazole; verapamil; fish oil; m...,None,High BP; Elevated cholesterol; GERD; No alcoho...,None
2,794359,2000-01-14,N,None,None,None,None,None,True,None,NaN,None,None,None,"Prescription: PLAQUENIL, FLORINEF, PRILOSEC, m...",None known,Lupus,None
3,794763,None,U,None,None,None,None,None,None,None,NaN,None,None,None,"Just vitamin C; zinc, B complex",Productive persistent cough; fatigue,None.,None
4,795410,2019-01-05,None,None,None,None,None,None,None,None,NaN,None,None,None,Lorsartan; amlodipine; multivitamin; B13,None,HTN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44339,839583,2019-10-03,N,None,None,None,True,None,None,None,NaN,None,None,None,"Synthroid, Zoloft, ASA, Preservision AREDS 2, ...",No illness at time of vaccination. No illness...,"HTN heart disease w/o failure, unspecified psy...",None
44340,798122,2019-01-18,Y,None,None,None,True,None,None,None,NaN,None,None,None,None,None,None,None
44341,843026,None,Y,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None
44342,827038,2019-08-02,N,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None


### Generate Primary Key for Symptom table. No other changes made to table as Symptom is an entity by itself.

In [7]:
%%bigquery
create table vaers_modeled.Symptom as
select ROW_NUMBER() OVER() as SYMPTOM_ID, VAERS_ID, SYMPTOM1, SYMPTOM2, SYMPTOM3, SYMPTOM4, SYMPTOM5 
from vaers_staging.Symptoms

""


In [8]:
%%bigquery
select * from vaers_modeled.Symptom

,SYMPTOM_ID,VAERS_ID,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,26,794308,Product administered to patient of inappropria...,None,None,None,None
1,31,794344,Exposure during pregnancy,None,None,None,None
2,44,794402,Pain,None,None,None,None
3,56,794493,Vertigo,None,None,None,None
4,270,795388,Adverse event,None,None,None,None
...,...,...,...,...,...,...,...
55255,54995,844760,Hypoaesthesia,Injected limb mobility decreased,Injection site pain,Paraesthesia,Product administered at inappropriate site
55256,55040,834189,Arthralgia,Immediate post-injection reaction,Musculoskeletal pain,Nausea,Pain
55257,55157,833481,Erythema,Extensive swelling of vaccinated limb,Injection site erythema,Injection site swelling,Injection site warmth
55258,55204,849460,Lymphocyte percentage decreased,Mean cell haemoglobin concentration normal,Mean cell haemoglobin normal,Mean cell volume normal,Mean platelet volume increased


### Split staging Vaccines table into separate entities: Vaccine and Manufacturer

#### Create Vaccine table

In [49]:
%%bigquery
create table vaers_modeled.Vaccine_interim as
select distinct VAX_NAME, VAX_TYPE
from vaers_staging.Vaccines

# create table vaers_modeled.Vaccine as
# select ROW_NUMBER() OVER(ORDER BY VAX_NAME) as VAX_ID, VAX_NAME, VAX_TYPE
# from vaers_staging.Vaccines

""


In [53]:
%%bigquery
create table vaers_modeled.Vaccine as
select ROW_NUMBER() OVER(ORDER BY VAX_NAME) as VAX_ID, VAX_NAME, VAX_TYPE
from vaers_modeled.Vaccine_interim

""


In [22]:
%%bigquery
alter table vaers_modeled.Vaccine
add VAX_ID INT


Executing query with job ID: 41eff8b0-18ca-4f3a-8ab4-71a4c3b5ec15
Query executing: 0.28s


ERROR:
 400 Syntax error: Expected keyword CHECK or keyword COLUMN or keyword CONSTRAINT or keyword FOREIGN but got identifier "VAX_ID" at [2:5]

(job ID: 41eff8b0-18ca-4f3a-8ab4-71a4c3b5ec15)

   -----Query Job SQL Follows-----    

    |    .    |    .    |    .    |
   1:alter table vaers_modeled.Vaccine
   2:add VAX_ID INT
    |    .    |    .    |    .    |


In [56]:
%%bigquery
select * from vaers_modeled.Vaccine

,VAX_ID,VAX_NAME,VAX_TYPE
0,28,HEP A (VAQTA),HEPA
1,45,INFLUENZA (H1N1) (H1N1 (MONOVALENT) (UNKNOWN)),FLUX(H1N1)
2,78,MENINGOCOCCAL CONJUGATE (MENVEO),MNQ
3,98,SMALLPOX (ACAM2000),SMALL
4,55,INFLUENZA (SEASONAL) (FLULAVAL QUADRIVALENT),FLU4
...,...,...,...
120,69,MEASLES + MUMPS + RUBELLA (MMR II),MMR
121,60,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),FLU3
122,86,"POLIO VIRUS, INACT. (NO BRAND NAME)",IPV
123,93,RABIES (RABIE-VAX),RAB


#### Create Manufacturer table

In [29]:
%%bigquery
create table vaers_modeled.Manufacturer as
select ROW_NUMBER() OVER(ORDER BY VAX_MANU) as MANU_ID, VAX_MANU
from vaers_staging.Vaccines

""


In [31]:
%%bigquery
select * from vaers_modeled.Manufacturer

,MANU_ID,VAX_MANU
0,28,"BERNA BIOTECH, LTD."
1,45,"BERNA BIOTECH, LTD."
2,78,CSL LIMITED
3,98,CSL LIMITED
4,134,CSL LIMITED
...,...,...
56267,56090,UNKNOWN MANUFACTURER
56268,56118,UNKNOWN MANUFACTURER
56269,56165,UNKNOWN MANUFACTURER
56270,56206,UNKNOWN MANUFACTURER


### Join attributes from staging Events and Vaccines to form new entity: Vaccination

In [55]:
%%bigquery
# create table vaers_modeled.Vaccination as
select ROW_NUMBER() OVER(ORDER BY e.VAERS_ID) as VACCINATION_ID, e.VAERS_ID, e.VAX_DATE, mv.VAX_ID, e.V_ADMINBY, e.V_FUNDBY, v.VAX_ROUTE, v.VAX_SITE
from (vaers_staging.Events e left join vaers_staging.Vaccines v on e.VAERS_ID = v.VAERS_ID)
    left join vaers_modeled.Vaccine mv on mv.VAX_NAME = v.VAX_NAME

,VACCINATION_ID,VAERS_ID,VAX_DATE,VAX_ID,V_ADMINBY,V_FUNDBY,VAX_ROUTE,VAX_SITE
0,1,794156,2018-10-29,49,PVT,None,IM,LA
1,2,794157,2018-12-28,124,PHM,None,None,None
2,3,794158,2018-12-30,124,PHM,None,None,None
3,4,794159,2018-12-27,18,PVT,None,IM,LL
4,5,794159,2018-12-27,72,PVT,None,SC,LL
...,...,...,...,...,...,...,...,...
56267,56268,852816,2019-12-06,84,OTH,UNK,IM,RA
56268,56269,853285,None,102,UNK,None,OT,None
56269,56270,854558,2019-10-14,82,PHM,None,IM,LA
56270,56271,854559,2019-10-16,49,WRK,None,IM,LA


In [54]:
%%bigquery
select v.VAX_NAME, VAX_ID
from vaers_staging.Vaccines v left join vaers_modeled.Vaccine mv on mv.VAX_NAME = v.VAX_NAME

,VAX_NAME,VAX_ID
0,HIB (ACTHIB),35
1,HIB (ACTHIB),35
2,HIB (ACTHIB),35
3,HIB (ACTHIB),35
4,HIB (ACTHIB),35
...,...,...
56267,MEASLES + MUMPS + RUBELLA + VARICELLA (PROQUAD),72
56268,MEASLES + MUMPS + RUBELLA + VARICELLA (NO BRAN...,71
56269,MEASLES + MUMPS + RUBELLA + VARICELLA (NO BRAN...,71
56270,MEASLES + MUMPS + RUBELLA + VARICELLA (NO BRAN...,71


### Identify PK for each modeled table

#### Patient table : PK is Patient_ID

In [2]:
%%bigquery
select count(*) as count from vaers_modeled.Report

,count
0,44344


In [4]:
%%bigquery
select count(distinct VAERS_ID) as count from vaers_modeled.Report

,count
0,44344


### Showing duplicates of VAERS_ID in Patient_Event and Vaccine_Event tables
##### As there are less distinct VAERS_ID values than the total number of rows, there exists duplicates of VAERS_ID
##### Also, given that the count of distinct VAERS_ID matches the count of PK above, we know that referential integrity is preserved

In [18]:
%%bigquery
select count(*) as count from vaers_modeled.Vaccine_Event

,count
0,56272


In [19]:
%%bigquery
select count(distinct VAERS_ID) as count from vaers_modeled.Vaccine_Event

,count
0,44344


In [20]:
%%bigquery
select count(*) as count from vaers_modeled.Patient_Event

,count
0,55260


In [21]:
%%bigquery
select count(distinct VAERS_ID) as count from vaers_modeled.Patient_Event

,count
0,44343


### Queries showing the duplicate entries. Explanations in TRANSFORMS.txt

In [24]:
%%bigquery
select v1.* from vaers_modeled.Vaccine_Event v1
join (SELECT VAERS_ID, COUNT(*)
FROM vaers_modeled.Vaccine_Event
group by VAERS_ID
HAVING count(*) > 1) v2
on v1.VAERS_ID = v2.VAERS_ID
order by v1.VAERS_ID
limit 10

,VAERS_ID,VAX_DATE,ONSET_DATE,V_ADMINBY,V_FUNDBY,PRIOR_VAX,VAX_TYPE,VAX_NAME,VAX_MANU,VAX_ROUTE,VAX_SITE
0,794159,2018-12-27,2018-12-28,PVT,None,None,MMRV,MEASLES + MUMPS + RUBELLA + VARICELLA (PROQUAD),MERCK & CO. INC.,SC,LL
1,794159,2018-12-27,2018-12-28,PVT,None,None,DTAPIPV,DTAP + IPV (KINRIX),GLAXOSMITHKLINE BIOLOGICALS,IM,LL
2,794162,2018-12-19,2018-12-26,PVT,None,None,HEPA,HEP A (HAVRIX),GLAXOSMITHKLINE BIOLOGICALS,IM,LL
3,794162,2018-12-19,2018-12-26,PVT,None,None,MMRV,MEASLES + MUMPS + RUBELLA + VARICELLA (PROQUAD),MERCK & CO. INC.,IM,RL
4,794165,2018-12-28,2018-12-28,PHM,None,None,PNC13,PNEUMO (PREVNAR13),PFIZER\WYETH,IM,RA
5,794165,2018-12-28,2018-12-28,PHM,None,None,FLU3,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),SANOFI PASTEUR,IM,LA
6,794170,2018-12-20,2018-12-30,PVT,None,None,VARCEL,VARICELLA (VARIVAX),MERCK & CO. INC.,IM,LL
7,794170,2018-12-20,2018-12-30,PVT,None,None,HEPA,HEP A (VAQTA),MERCK & CO. INC.,IM,RL
8,794170,2018-12-20,2018-12-30,PVT,None,None,MMR,MEASLES + MUMPS + RUBELLA (MMR II),MERCK & CO. INC.,IM,LL
9,794178,2018-11-14,2018-11-29,MIL,None,None,SMALL,SMALLPOX (ACAM2000),SANOFI PASTEUR,None,LA


In [26]:
%%bigquery
select p1.* from vaers_modeled.Patient_Event p1
join (SELECT VAERS_ID, COUNT(*)
FROM vaers_modeled.Patient_Event
group by VAERS_ID
HAVING count(*) > 1) p2
on p1.VAERS_ID = p2.VAERS_ID
order by p1.VAERS_ID
limit 10

,VAERS_ID,STATE,AGE_YRS,SEX,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,...,DISABLE,OTHER_MEDS,CUR_ILL,BIRTH_DEFECT,ALLERGIES,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
0,794157,IL,68.0,F,None,None,None,None,None,None,...,None,"Levothyroxine, sertraline, vitamin B complex, ...",None,None,None,Apathy,Arthralgia,Asthenia,Injection site erythema,Injection site pain
1,794157,IL,68.0,F,None,None,None,None,None,None,...,None,"Levothyroxine, sertraline, vitamin B complex, ...",None,None,None,Injection site pruritus,Injection site swelling,Injection site warmth,Listless,Night sweats
2,794160,TX,79.0,F,None,None,None,None,None,None,...,None,ATORVASTATIN; CELEBREX; XYZAL; D-3; ELIQUIS; A...,SINUSITIS; ATHROSCLEROSIS; DIABETES; HYPERCHOL...,None,ACE INHIBITORS,Asthenia,Chills,Fatigue,Influenza like illness,Injection site erythema
3,794160,TX,79.0,F,None,None,None,None,None,None,...,None,ATORVASTATIN; CELEBREX; XYZAL; D-3; ELIQUIS; A...,SINUSITIS; ATHROSCLEROSIS; DIABETES; HYPERCHOL...,None,ACE INHIBITORS,Injection site swelling,Lip blister,Lip swelling,Pain,None
4,794163,MA,65.0,F,None,None,None,None,None,None,...,None,"Extra strength arthritis Tylenol, Simvastatin,...",None,None,"Chocolate, nuts, legumes",Myalgia,Nausea,Pyrexia,None,None
5,794163,MA,65.0,F,None,None,None,None,None,None,...,None,"Extra strength arthritis Tylenol, Simvastatin,...",None,None,"Chocolate, nuts, legumes",Abdominal pain,Arthralgia,Dizziness,Fatigue,Headache
6,794168,MA,43.0,M,None,None,None,None,None,None,...,None,probiotics; magnesium,None,None,NKDA,Vomiting,None,None,None,None
7,794168,MA,43.0,M,None,None,None,None,None,None,...,None,probiotics; magnesium,None,None,NKDA,Dizziness,Ear pain,Labyrinthitis,Migraine,Nausea
8,794168,MA,43.0,M,None,None,None,None,None,None,...,None,probiotics; magnesium,None,None,NKDA,Nuclear magnetic resonance imaging normal,Tinnitus,Vertigo,Vestibular migraine,Viral infection
9,794175,None,66.0,F,None,None,None,None,None,None,...,None,none,none,None,none,Injected limb mobility decreased,Injection site pain,Loss of consciousness,None,None
